In [1]:
%load_ext autoreload
%load_ext line_profiler
%autoreload 2
%matplotlib ipympl

import sys
import numpy as np
import time

sys.path.append('./src')
verbose = False

random_seed = np.random.randint(100)
random_seed

3

In [2]:
from src.utils_solver import Lmatrix2paths, adapted_empirical_measure, adapted_wasserstein_squared, quantization, nested, plot_V

n_sample = 1000
normalize = False

print("mu")
L = np.array([[1, 0, 0], [2, 4, 0], [3, 2, 1]])
X,A = Lmatrix2paths(L, n_sample, seed = random_seed)

print("nu")
M = np.array([[1, 0, 0], [2, 3, 0], [3, 1, 2]])
Y,B = Lmatrix2paths(M, n_sample, seed = random_seed)

mu
Cholesky:
[[1 0 0]
 [2 4 0]
 [3 2 1]]
Covariance:
[[ 1  2  3]
 [ 2 20 14]
 [ 3 14 14]]
nu
Cholesky:
[[1 0 0]
 [2 3 0]
 [3 1 2]]
Covariance:
[[ 1  2  3]
 [ 2 13  9]
 [ 3  9 14]]


In [3]:
from adapted_empirical_measure.AEM_grid import uniform_empirical_grid_measure
from trees.Build_trees_from_paths import build_tree_from_paths
from trees.TreeAnalysis import get_depth
from awd_trees.Nested_Dist_Algo import compute_nested_distance


In [4]:
# Compute uniform adapted empirical grid measures with weights
adapted_X, adapted_weights_X = uniform_empirical_grid_measure(X.T, delta_n=0.1, use_weights=True)
adapted_Y, adapted_weights_Y = uniform_empirical_grid_measure(Y.T, delta_n=0.1, use_weights=True)

# Build trees from the adapted paths
adapted_tree_1 = build_tree_from_paths(adapted_X, adapted_weights_X)
adapted_tree_2 = build_tree_from_paths(adapted_Y, adapted_weights_Y)

# Compute the nested (adapted optimal transport) distance and measure execution time
max_depth = get_depth(adapted_tree_1)
start_time = time.perf_counter()
distance_pot = compute_nested_distance(
    adapted_tree_1,
    adapted_tree_2,
    max_depth,
    method="solver_lp",
    return_matrix=False,
    lambda_reg=0,
    power=2,
)
end_time = time.perf_counter()
elapsed_time_pot = end_time - start_time

print("Numerical AW_2^2 (Adapted OT):", distance_pot)
print("Elapsed time (Adapted OT): {:.4f} seconds".format(elapsed_time_pot))

Depth 0: 100%|██████████| 1/1 [00:00<00:00, 131.45it/s]

Numerical AW_2^2 (Adapted OT): 3.1520299999999986
Elapsed time (Adapted OT): 235.6926 seconds


In [5]:
adaptedX = adapted_empirical_measure(X, delta_n = 0.1)
adaptedY = adapted_empirical_measure(Y, delta_n = 0.1)

q2v, v2q, mu_x, nu_y, q2v_x, v2q_x, q2v_y, v2q_y = quantization(adaptedX, adaptedY, markovian=False)

start_time = time.perf_counter()
AW_2square, V = nested(mu_x, nu_y, v2q_x, v2q_y, q2v, markovian=False)
end_time = time.perf_counter()

dist_bench = adapted_wasserstein_squared(A, B)
print("Theoretical AW_2^2: ", dist_bench)
print("Numerical AW_2^2: ", AW_2square)
print("Elapsed time (Adapted OT): {:.4f} seconds".format(elapsed_time_pot))

Quantization ......
Number of distinct values in global quantization:  234
Number of condition subpaths of mu_x
Time 0: 1
Time 1: 56
Time 2: 901
Number of condition subpaths of nu_y
Time 0: 1
Time 1: 56
Time 2: 876
Nested backward induction .......


Timestep 0: 100%|██████████| 1/1 [00:00<00:00, 1080.73it/s]

Theoretical AW_2^2:  3.0
Numerical AW_2^2:  3.1520300000000003
Elapsed time (Adapted OT): 235.6926 seconds


In [7]:
from scipy.optimize import linprog

def solver_lp(distance_matrix_subset, pi_ratios, pi_tilde_ratios):
    n, m = distance_matrix_subset.shape
    c = distance_matrix_subset.flatten()

    # Row constraints
    A_eq = np.zeros((n + m, n * m))
    b_eq = np.concatenate([pi_ratios, pi_tilde_ratios])

    for i in range(n):  # Row constraints
        A_eq[i, i * m : (i + 1) * m] = 1

    for j in range(m):  # Column constraints
        A_eq[n + j, j::m] = 1

    res = linprog(c, A_eq=A_eq, b_eq=b_eq, method="highs")

    return res.x.reshape(n, m) if res.success else None

In [33]:
n = 100
distance_matrix_subset = np.random.normal(size = [n,n])
pi_ratios = np.ones(n)/n
pi_tilde_ratios = np.ones(n)/n

In [42]:
start_time = time.perf_counter()
sol = solver_lp(distance_matrix_subset, pi_ratios, pi_tilde_ratios)
end_time = time.perf_counter()
elapsed_time = end_time - start_time
print("Elapsed time (Adapted OT): {:.4f} seconds".format(elapsed_time))

Elapsed time (Adapted OT): 0.0359 seconds
